In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import datetime

In [ ]:
df = pd.read_csv('dataset_finalv3.csv').dropna(subset=['feedback_rating'])
df = df.query('feedback_rating > 0')
df.reset_index(inplace=True)
df.drop(columns='index',inplace=True)

In [ ]:
pd.set_option('display.max_columns', 999)

In [ ]:
#transformando a coluna da data de correção em date time
df.start = pd.to_datetime(df.start)


In [ ]:
#criando uma coluna que diz o ano da correção
df['year'] = [df.loc[i].start.year for i in range(len(df))]


In [ ]:
#criando outro dataframe com dados apenas de 2020 pra cima
df2020 = df.query('year >= 2020')
df2020.reset_index(inplace=True)
df2020.drop(columns='index',inplace=True)

C:\Users\luizf\AppData\Local\Temp\ipykernel_11820\1715257305.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2020.drop(columns='index',inplace=True)


In [ ]:
df2020.dropna(inplace=True)
df2020.reset_index(inplace=True)
df2020.drop(columns=['index'],inplace=True)

C:\Users\luizf\AppData\Local\Temp\ipykernel_11820\3482070822.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2020.dropna(inplace=True)
C:\Users\luizf\AppData\Local\Temp\ipykernel_11820\3482070822.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2020.drop(columns=['index'],inplace=True)


In [ ]:
#transformando a variável alvo para classificação
df2020.feedback_rating = np.where(df2020.feedback_rating > 3, 'satisfeito', 'insatisfeito')

C:\Users\luizf\AppData\Local\Temp\ipykernel_11820\2878062067.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2020.feedback_rating = np.where(df2020.feedback_rating > 3, 'satisfeito', 'insatisfeito')


In [ ]:
df2020.feedback_rating = df2020.feedback_rating.map({'insatisfeito':1,'satisfeito':0})

C:\Users\luizf\AppData\Local\Temp\ipykernel_11820\1324156261.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2020.feedback_rating = df2020.feedback_rating.map({'insatisfeito':1,'satisfeito':0})


In [ ]:
#instalando a biblioteca que fornece os encoders
!pip install category_encoders 

In [ ]:
train = df2020[(df2020.start < datetime.datetime(2021,11,1))].drop(columns=['start','year','oficial_feedback_revis'])
test = df2020[~(df2020.start < datetime.datetime(2021,11,1))].drop(columns=['start','year','oficial_feedback_revis'])

In [ ]:
#aplicando o encoder nas colunas categóricas

import category_encoders as ce 
# parâmetro cols identifica as colunas categóricas
js_treino = ce.JamesSteinEncoder(cols=['tipo_cliente','formato_redacao', 'dia_semana',
                                       'mes_ano','dia_mes','theme_id_essay',
                                       ], model='beta')



treino = js_treino.fit_transform(train.drop(columns='feedback_rating'), train.feedback_rating)

teste = js_treino.transform(test.drop(columns='feedback_rating'))


treino = pd.concat([treino,train.feedback_rating],axis=1)

teste = pd.concat([teste,test.feedback_rating],axis=1)

In [ ]:
X_train2 = treino.drop(columns='feedback_rating')
y_train2 = treino.feedback_rating

X_test2020s = teste.drop(columns='feedback_rating')
y_test2020s = teste.feedback_rating

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state = 7)

X_nof2020sm, y_nof2020sm = sm.fit_resample(X_train2, y_train2)

In [ ]:
gb = GradientBoostingClassifier(random_state=7)

In [ ]:
from sklearn.feature_selection import RFE
results = {}
for i in range(len(X_nof2020sm.columns),1,-1):
  print(f'{i} feature(s)')
  selector = RFE(gb,n_features_to_select=i)
  selector = selector.fit(X_nof2020sm, y_nof2020sm)
  ypred_gb = selector.predict(X_test2020s)
  results[i] = accuracy_score(y_test2020s,ypred_gb)

46 feature(s)
45 feature(s)
44 feature(s)
43 feature(s)
42 feature(s)
41 feature(s)
40 feature(s)
39 feature(s)
38 feature(s)
37 feature(s)
36 feature(s)
35 feature(s)
34 feature(s)
33 feature(s)
32 feature(s)
31 feature(s)
30 feature(s)
29 feature(s)
28 feature(s)
27 feature(s)
26 feature(s)
25 feature(s)
24 feature(s)
23 feature(s)
22 feature(s)
21 feature(s)
20 feature(s)
19 feature(s)
18 feature(s)
17 feature(s)
16 feature(s)
15 feature(s)
14 feature(s)
13 feature(s)
12 feature(s)
11 feature(s)
10 feature(s)
9 feature(s)
8 feature(s)
7 feature(s)
6 feature(s)
5 feature(s)
4 feature(s)
3 feature(s)
2 feature(s)


In [ ]:
results

{46: 0.8063291139240506,
 45: 0.8063291139240506,
 44: 0.8063291139240506,
 43: 0.8063291139240506,
 42: 0.8063291139240506,
 41: 0.8063291139240506,
 40: 0.8063291139240506,
 39: 0.805379746835443,
 38: 0.8066455696202531,
 37: 0.8056962025316455,
 36: 0.8060126582278481,
 35: 0.8047468354430379,
 34: 0.8037974683544303,
 33: 0.8037974683544303,
 32: 0.8047468354430379,
 31: 0.8031645569620253,
 30: 0.8044303797468354,
 29: 0.8034810126582278,
 28: 0.8034810126582278,
 27: 0.8034810126582278,
 26: 0.8037974683544303,
 25: 0.805379746835443,
 24: 0.8041139240506329,
 23: 0.8044303797468354,
 22: 0.8044303797468354,
 21: 0.8006329113924051,
 20: 0.8015822784810127,
 19: 0.8025316455696202,
 18: 0.7996835443037975,
 17: 0.8,
 16: 0.7981012658227848,
 15: 0.7981012658227848,
 14: 0.7955696202531646,
 13: 0.7939873417721519,
 12: 0.7920886075949367,
 11: 0.7917721518987342,
 10: 0.7870253164556962,
 9: 0.790506329113924,
 8: 0.7914556962025316,
 7: 0.785759493670886,
 6: 0.7800632911392406

In [ ]:
selector2 = RFE(gb,n_features_to_select=38,verbose = 1)
selector2 = selector2.fit(X_nof2020sm, y_nof2020sm)

Fitting estimator with 46 features.
Fitting estimator with 45 features.
Fitting estimator with 44 features.
Fitting estimator with 43 features.
Fitting estimator with 42 features.
Fitting estimator with 41 features.
Fitting estimator with 40 features.
Fitting estimator with 39 features.


In [ ]:
colunas = []
for i in range(len(X_nof2020sm.columns)):
  if selector2.support_[i] == True:
    print(f'{X_nof2020sm.columns[i]}:{selector2.support_[i]}')
    colunas.append(X_nof2020sm.columns[i])

tipo_cliente:True
tamanho_comentario_geral:True
correction_time:True
proporcao_positiva:True
qtd_comentarios_especificos:True
proporcao_negativa:True
qtd_comentarios_positivos:True
qtd_comentarios_negativos:True
tamanho_medio_comentarios_especificos:True
tempo_vinculo_corretor:True
qtd_correcoes_anteriores:True
tempo_espera:True
mes_ano:True
dia_mes:True
tempo_vinculo_aluno:True
theme_id_essay:True
competencia1:True
competencia2:True
competencia3:True
nota_final:True
media_temporal_aluno:True
media_temporal_corretor:True
std_temporal_aluno:True
std_temporal_corretor:True
mediana_temporal_aluno:True
mediana_temporal_corretor:True
score_medio_temporal_aluno:True
score_medio_temporal_corretor:True
score_std_temporal_aluno:True
qtd_feedbacks_aluno:True
feedbacks_positivos_aluno:True
feedbacks_negativos_aluno:True
proporcao_feedback_positivo_aluno:True
proporcao_feedback_negativo_aluno:True
qtd_feedbacks_corretor:True
feedbacks_positivos_corretor:True
proporcao_feedback_positivo_corretor:Tr

In [ ]:
colunas

['tipo_cliente',
 'tamanho_comentario_geral',
 'correction_time',
 'proporcao_positiva',
 'qtd_comentarios_especificos',
 'proporcao_negativa',
 'qtd_comentarios_positivos',
 'qtd_comentarios_negativos',
 'tamanho_medio_comentarios_especificos',
 'tempo_vinculo_corretor',
 'qtd_correcoes_anteriores',
 'tempo_espera',
 'mes_ano',
 'dia_mes',
 'tempo_vinculo_aluno',
 'theme_id_essay',
 'competencia1',
 'competencia2',
 'competencia3',
 'nota_final',
 'media_temporal_aluno',
 'media_temporal_corretor',
 'std_temporal_aluno',
 'std_temporal_corretor',
 'mediana_temporal_aluno',
 'mediana_temporal_corretor',
 'score_medio_temporal_aluno',
 'score_medio_temporal_corretor',
 'score_std_temporal_aluno',
 'qtd_feedbacks_aluno',
 'feedbacks_positivos_aluno',
 'feedbacks_negativos_aluno',
 'proporcao_feedback_positivo_aluno',
 'proporcao_feedback_negativo_aluno',
 'qtd_feedbacks_corretor',
 'feedbacks_positivos_corretor',
 'proporcao_feedback_positivo_corretor',
 'proporcao_feedback_negativo_corr